In [1]:
from numba import njit
import random
from santorinai.player import Player
import time
import numpy as np
from numba.typed import List

Basic Rules

In [2]:
@njit(cache=True)
def movesPlayer(board, players, x, y):
    z = board[x, y]
    moves = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= board.shape[0]:
                continue
            if y+j < 0 or y+j >= board.shape[1]:
                continue
            if board[x+i, y+j] - z < 2 and board[x+i, y+j] < 4 and (x+i, y+j) not in players:
                moves.append((x+i, y+j))
    return moves

In [3]:
@njit(cache=True)
def constructsPlayer(board, players, x, y):
    places = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= board.shape[0]:
                continue
            if y+j < 0 or y+j >= board.shape[1]:
                continue
            if board[x+i, y+j] < 4 and (x+i, y+j) not in players:
                places.append((x+i, y+j))
    return places

In [4]:
@njit(cache=True)
def win(board, x, y):
    return board[x, y] == 3

Minimax

In [5]:
@njit(cache=True)
def minimax(board, players, playerAct, depth, alpha, beta, maximizingPlayer):
    existAction = False
    if win(board, *players[(playerAct-1) % 4]):
        return -1 if maximizingPlayer else 1
    if depth == 0:
        return 0
    for move in movesPlayer(board, players, *players[playerAct]):
        newPlayers = players.copy()
        newPlayers[playerAct] = move
        for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
            if not existAction:
                existAction = True
            newBoard = np.copy(board)
            newBoard[construct[0], construct[1]] += 1
            value = minimax(newBoard, newPlayers, (playerAct+1) % 4, depth-1, alpha, beta, not maximizingPlayer)
            if maximizingPlayer:
                alpha = max(alpha, value)
                if alpha >= beta:
                    return alpha
            else:
                beta = min(beta, value)
                if beta <= alpha:
                    return beta
    
    if not existAction:  # Pawn is blocked
        return minimax(board, players, (playerAct+1) % 4, depth-1, alpha, beta, not maximizingPlayer)
    
    return alpha if maximizingPlayer else beta

In [6]:
@njit(cache=True)
def getBestMove(board, players, playerAct, depth):
    bestValue = -2
    bestMove = None
    bestConstruct = None
    while bestMove is None:
        for move in movesPlayer(board, players, *players[playerAct]):
            newPlayers = players.copy()
            newPlayers[playerAct] = move
            for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
                if depth == 0:
                    return move, construct
                newBoard = np.copy(board)
                newBoard[construct[0], construct[1]] += 1
                value = minimax(newBoard, newPlayers, (playerAct+1) % 4, depth-1, bestValue, 1, False)
                if value is not None and value > bestValue:
                    bestValue = value
                    bestMove = move
                    bestConstruct = construct
                    if bestValue >= 1:
                        return bestMove, bestConstruct
        depth -= 1
    return bestMove, bestConstruct

In [7]:
import threading
import time

class threadWithReturn(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(threadWithReturn, self).__init__(*args, **kwargs)
        self._return = None
    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args, **self._kwargs)
            
def playProgressive(board, players, playerAct, depth):
    time_start = time.time()
    act_depth = 6
    while act_depth <= depth:
        thread = threadWithReturn(target=getBestMove, args=(board, players, playerAct, act_depth))
        thread.start()
        while time.time() - time_start <= 5 and thread.is_alive():
            time.sleep(0.1)
        if time.time() - time_start > 5:
            print(act_depth - 1)
            return move, construct
        act_depth += 1
        move, construct = thread._return
    print(depth)
    return move, construct

In [8]:
class TktBot4(Player):
    """
    Minimax + Monte Carlo Tree Search bot
    """

    def name(self):
        return "Tkt bot 4"

    # Placement of the pawns
    def place_pawn(self, board, pawn):
        my_choice = (2, 2) if (2, 2) in board.get_possible_movement_positions(
            pawn) else random.choice(board.get_possible_movement_positions(pawn))
        return my_choice

    # Movement and building
    def play_move(self, board, pawn):
        return playProgressive(
                np.array(board.board), List([player.pos for player in board.pawns]), pawn.number - 1, 20)

Test

In [1]:
from santorinai.tester import Tester
from TktBot3 import TktBot3
from TktBot4 import TktBot4

tester = Tester()
tester.delay_between_moves = 0
tester.display_board = False
tester.verbose_level = 2

p1 = TktBot4()
p2 = TktBot3()

tester.play_1v1(p1, p2, nb_games = 20)

Game 1
Player Tkt bot 3 is placing pawn 1
   Pawn placed at position (2, 2)
Player Tkt bot 4 is placing pawn 2
   Pawn placed at position (4, 1)
Player Tkt bot 3 is placing pawn 3
   Pawn placed at position (0, 4)
Player Tkt bot 4 is placing pawn 4
   Pawn placed at position (0, 1)

Playing the game
   Current pawn: pawn 1 of player 1 at position (2, 2)
Player Tkt bot 4 is moving pawn 1
6
   Pawn moved at position (1, 1) and built at position (0, 0)

30 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
40 10 _0 _0 20 
_1 _0 _0 _0 _0 

   Current pawn: pawn 2 of player 2 at position (4, 1)
Player Tkt bot 3 is moving pawn 2
6
   Pawn moved at position (3, 0) and built at position (2, 0)

30 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
40 10 _0 _0 _0 
_1 _0 _1 20 _0 

   Current pawn: pawn 3 of player 1 at position (0, 4)
Player Tkt bot 4 is moving pawn 3
8
   Pawn moved at position (0, 3) and built at position (0, 2)

_0 _0 _0 _0 _0 
30 _0 _0 _0 _0 
_1 _0 _0 _0 _0 
40 10 _0 _0 _0 
_1 _0 _1 20

KeyboardInterrupt: 

In [20]:
import TktBot3 as first
import TktBot4 as second
import numpy as np

In [6]:
board = np.array([[0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,1,0], [0,0,0,0,0]])

players = first.List([(3, 3), (1, 1), (1, 3), (3, 1)])

In [23]:
%timeit first.getBestMove(board, players, 0, 8)

4.01 s ± 49.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit second.getBestMove(board, players, 0, 8)